In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# standard imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plte
%matplotlib inline
import tensorflow as tf
import tensorflow_hub as hub


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Je télécharge ma donnée et la regarde en détail

In [ ]:
labels_csv = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
print(labels_csv.describe())
print(labels_csv.head())

Je peux voir que j'ai 120 races de chien pour 10 222 photos. 

In [ ]:
labels_csv["breed"].value_counts().plot.bar(figsize=(20, 5));

Dans le schéma ci-dessus, on peut observer qu'il y a plus de 60 images pour chaque race de chien. C'est une bonne quantité, car pour certains de leurs produits de vision, Google recommande un minimum de 10 images par classe pour commencer. 

Il est temps de récupérer les images et leurs étiquettes

Puisque nous avons les ID des images et leurs étiquettes dans un DataFrame (labels_csv), nous allons l'utiliser pour créer :

* 1) Une liste de chemins d'accès aux images d'entraînement
* 2) Un tableau de toutes les étiquettes
* 3) Un tableau de toutes les étiquettes uniques

Nous allons seulement créer une liste de chemins d'accès aux images plutôt que de les importer toutes pour commencer. En effet, il est beaucoup plus efficace de travailler avec des chemins de fichiers (chaînes de caractères) qu'avec des images.

Comme nous l'utiliserons à plusieurs reprises, nous enregistrerons le chemin d'accès à nos fichiers d'entraînement dans une variable train_path.

In [ ]:
# Define our training file path 
train_path = "../input/dog-breed-identification/train/"

In [ ]:
# Create pathnames from image ID's
filenames = [train_path + fname + ".jpg" for fname in labels_csv["id"]]

# Check the first 10 filenames
filenames[:10]

Maintenant que nous avons une liste de tous les noms de fichiers de la colonne ID de labels_csv, nous pouvons la comparer au nombre de fichiers dans notre répertoire de données d'entraînement pour voir s'ils correspondent.

In [ ]:
if len(os.listdir('/kaggle/input/dog-breed-identification/train/')) == len(filenames):
    print('Number of file matches number of actual images!')
else:
    print('Number of file doesnot matches number of actual images!!')

In [ ]:
#Je retrouve une image par son id
labels_csv['breed'][900]

In [ ]:
# Je regarde l'image directement
from IPython.display import display, Image
Image(filenames[900])

Maintenant que nous avons rassemblé les chemins de fichiers des images, récupérons les étiquettes.
Nous allons les prendre dans labels_csv et les transformer en tableau NumPy.

In [ ]:
import numpy as np
labels = labels_csv["breed"].to_numpy() # convert labels column to NumPy array
labels[:10]

Comparons le nombre d'étiquettes au nombre de noms de fichiers.

In [ ]:
if len(labels) == len(filenames):
    print('Number of labels matches the number of filenames.')
else:
    print('Number of labels doesnot matches the number of filenames')

Si tout a fonctionné, nous devrions avoir la même quantité d'images et d'étiquettes.

Enfin, comme un modèle d'apprentissage automatique ne peut pas prendre de chaînes de caractères en entrée (ce que sont les étiquettes actuellement), nous devons **convertir nos étiquettes en chiffres.**

Pour commencer, nous allons trouver tous les **noms uniques de races de chiens.**

Ensuite, nous allons **parcourir la liste des étiquettes, les comparer aux races uniques** et **créer une liste de booléens** indiquant laquelle est l'étiquette réelle (Vrai) et lesquelles ne le sont pas (Faux).

In [ ]:
unique_breed = np.unique(labels) 
len(unique_breed)

La longueur de unique_breeds doit être de 120, ce qui signifie que nous travaillons avec des images de 120 races de chiens différentes.

Utilisez maintenant unique_breeds pour transformer notre tableau de labels en un tableau de booléens.

In [ ]:
#exemple d'une étiquette transformée en tableau de boléens
print(labels[0])
labels[0] == unique_breed

In [ ]:
# Turn every label into a boolean array
boolean_labels = [label == np.array(unique_breed) for label in labels]
boolean_labels[:2]

un concept important de l'apprentissage automatique consiste à convertir vos données en chiffres avant de les transmettre à un modèle d'apprentissage automatique.

Dans ce cas, nous avons transformé un seul nom de race de chien, tel que boston_bull, en un tableau de chiffres sur le principe du One Hot Encoder.

In [ ]:
# Turining boolean arrays into integers.
print(labels[0])   #orginal index
print(np.where(unique_breed==labels[0]))    #index where labels occurs.
print(boolean_labels[0].argmax())     #index where label occurs in boolean array
print(boolean_labels[0].astype(int))   #there will be a 1 where sample label occurs

Maintenant que nous avons nos étiquettes au format numérique et que les chemins de fichier de nos images sont facilement accessibles (ils ne sont pas encore numériques), nous allons diviser nos données.

# Création de notre propre ensemble de validation

Puisque l'ensemble de données de Kaggle n'est pas fourni avec un ensemble de validation (une division des données sur laquelle nous pouvons tester notre modèle avant de faire des prédictions finales sur l'ensemble de test), nous allons en créer un.

Nous pouvons utiliser la fonction train_test_split de Scikit-Learn ou nous pouvons simplement faire des divisions manuelles des données.

Pour des raisons d'accessibilité ultérieure, sauvegardons nos noms de fichiers en X (données) et nos étiquettes en y.

In [ ]:
# Setup X & y variables
X = filenames
y = boolean_labels

print(f"Number of training images: {len(X)}")
print(f"Number of labels: {len(y)}")

Étant donné que nous travaillons avec plus de 10 000 images, il est préférable de travailler avec une partie d'entre elles pour s'assurer que tout fonctionne bien avant de s'entraîner sur toutes les images.

En effet, le calcul de plus de 10 000 images peut prendre beaucoup de temps. Et notre objectif, lorsque nous travaillons sur des projets d'apprentissage automatique, est de réduire le temps entre les expériences.

Commençons à expérimenter avec 1000 et augmentons ce nombre selon nos besoins.

In [ ]:
#set number of images to set for the experiment.
num_img = 1000 #@param {type:"slider",min:1000,max:10000}

Divisons maintenant nos données en ensembles de formation et de validation. Nous utiliserons une répartition 80/20 (80% de données de formation, 20% de données de validation).

In [ ]:
#let's split our data into train and validation.
from sklearn.model_selection import train_test_split

#spliting into training and validation of total size NUM_IMAGES.

X_train,X_val,y_train,y_val = train_test_split(X[:num_img],
                                                y[:num_img],
                                                test_size=0.2,
                                                random_state=42)
len(X_train),len(X_val),len(y_train),len(y_val)

In [ ]:
# Check out the training data (image file paths and labels)
X_train[:1], y_train[:1]

# Prétraitement des images (transformation des images en tenseurs)


Nos étiquettes sont au format numérique mais nos images ne sont encore que des chemins de fichiers.

Comme nous utilisons TensorFlow, **nos données doivent être sous la forme de tenseurs.**

Un tenseur est un moyen de **représenter des informations sous forme de nombres.** .Un tenseur peut être considéré comme une combinaison de tableaux NumPy, mais avec la capacité spéciale d'être utilisé sur un GPU.

En raison de la façon dont TensorFlow stocke les informations (dans les tenseurs), il permet aux modèles d'apprentissage automatique et d'apprentissage profond d'être **exécutés sur des GPU (généralement plus rapides pour le calcul numérique).**

Pour **prétraiter nos images en Tensors**, nous allons écrire une fonction qui fait plusieurs choses :

* Prend un nom de fichier d'image en entrée.
* Utilise TensorFlow pour lire le fichier et l'enregistrer dans une variable, image.
* Transformer notre image (un fichier jpeg) en tenseurs.
* Redimensionne l'image pour qu'elle soit de forme (224, 224).
* Retourner l'image modifiée.

Un bon endroit pour lire sur ce type de fonction est la documentation TensorFlow sur le **chargement des images.**

Vous vous demandez peut-être pourquoi **(224, 224)**, qui correspond à **(hauteur, largeur)**. C'est parce que c'est la taille de l'entrée que notre modèle (nous le verrons bientôt) prend, une image qui est (224, 224, 3).

**Le 3, c'est le nombre de canaux de couleur par pixel, rouge, vert et bleu (RVB).**

Rendons cela un peu plus concret.

In [ ]:
# Convert image to NumPy array
from matplotlib.pyplot import imread
image = imread(filenames[42]) # read in an image
image.shape

Remarquez la forme de l'image. C'est (257, 350, 3). C'est la hauteur, la largeur, la valeur du canal de couleur.

Et vous pouvez facilement le convertir en un tenseur en utilisant tf.constant().

In [ ]:
tf.constant(image)[:2]

In [ ]:
# Define image size
IMG_SIZE = 224

def process_image(image_path):
  """
  Takes an image file path and turns it into a Tensor.
  """
  # Read in image file
  image = tf.io.read_file(image_path)
  # Turn the jpeg image into numerical Tensor with 3 colour channels (Red, Green, Blue)
  image = tf.image.decode_jpeg(image, channels=3)
  # Convert the colour channel values from 0-225 values to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Resize the image to our desired size (224, 244)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
  return image

In [ ]:
# Create a simple function to return a tuple (image, label)
def get_image_label(image_path, label):
  """
  Takes an image file path name and the associated label,
  processes the image and returns a tuple of (image, label).
  """
  image = process_image(image_path)
  return image, label

# Création de lots de données
Maintenant que nous avons une fonction pour convertir nos images en tenseurs, nous allons en construire une fonction pour **transformer nos données en lots** (un TensorFlow BatchDataset).

Qu'est-ce qu'un batch ?

Un lot (également appelé mini-batch) est une petite portion de vos données, disons 32 (32 est généralement la taille de lot par défaut) images et leurs étiquettes. En apprentissage profond, au lieu de trouver des modèles dans un ensemble de données entier en même temps, vous les trouvez souvent un lot à la fois.

Disons que vous avez affaire à plus de 10 000 images (ce qui est le cas). Ensemble, ces fichiers peuvent occuper plus de mémoire que votre GPU. Essayer de les calculer tous entraînerait une erreur.

Il est donc plus efficace de  **créer des lots plus petits de vos données et de les calculer un par un.**

TensorFlow est très efficace lorsque vos données sont des lots de tenseurs (image, étiquette). Nous allons donc construire une fonction pour les créer en premier. Nous allons profiter de la fonction process_image en même temps.

In [ ]:
# Create a simple function to return a tuple (image, label)
def get_image_label(image_path, label):
  """
  Takes an image file path name and the associated label,
  processes the image and returns a tuple of (image, label).
  """
  image = process_image(image_path)
  return image, label

Maintenant que nous disposons d'une fonction simple pour transformer les noms des chemins d'accès aux fichiers d'images et les étiquettes associées en tuples (nous les transformerons ensuite en tenseurs), nous allons créer une fonction pour créer des lots de données.

Puisque nous aurons affaire à trois ensembles de données différents (formation, validation et test), nous nous assurerons que la fonction peut s'adapter à chaque ensemble.

Nous définirons une taille de lot par défaut de 32.

In [ ]:
# Define the batch size, 32 is a good default
BATCH_SIZE = 32

# Create a function to turn data into batches
def create_data_batches(x, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
  """
  Creates batches of data out of image (x) and label (y) pairs.
  Shuffles the data if it's training data but doesn't shuffle it if it's validation data.
  Also accepts test data as input (no labels).
  """
  # If the data is a test dataset, we probably don't have labels
  if test_data:
    print("Creating test data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x))) # only filepaths
    data_batch = data.map(process_image).batch(BATCH_SIZE)
    return data_batch
  
  # If the data if a valid dataset, we don't need to shuffle it
  elif valid_data:
    print("Creating validation data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                               tf.constant(y))) # labels
    data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch

  else:
    # If the data is a training dataset, we shuffle it
    print("Creating training data batches...")
    # Turn filepaths and labels into Tensors
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                              tf.constant(y))) # labels
    
    # Shuffling pathnames and labels before mapping image processor function is faster than shuffling images
    data = data.shuffle(buffer_size=len(x))

    # Create (image, label) tuples (this also turns the image path into a preprocessed image)
    data = data.map(get_image_label)

    # Turn the data into batches
    data_batch = data.batch(BATCH_SIZE)
  return data_batch

In [ ]:
# Create training and validation data batches
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

In [ ]:
# Check out the different attributes of our data batches
train_data.element_spec, val_data.element_spec

Nous avons nos données en lots, plus précisément, elles sont dans des paires Tensor de (images, étiquettes) prêtes à être utilisées sur un GPU.

# Visualisation des lots de données

In [ ]:
import matplotlib.pyplot as plt

# Create a function for viewing images in a data batch
def show_25_images(images, labels):
  """
  Displays 25 images from a data batch.
  """
  # Setup the figure
  plt.figure(figsize=(10, 10))
  # Loop through 25 (for displaying 25 images)
  for i in range(25):
    # Create subplots (5 rows, 5 columns)
    ax = plt.subplot(5, 5, i+1)
    # Display an image
    plt.imshow(images[i])
    # Add the image label as the title
    plt.title(unique_breed[labels[i].argmax()])
    # Turn gird lines off
    plt.axis("off")

Pour rendre le calcul efficace, un lot est une collection de tenseurs fortement enroulée.
Ainsi, pour visualiser les données d'un lot, nous devons le dérouler. Nous pouvons le faire en appelant la méthode as_numpy_iterator() sur un lot de données. Cela transformera notre lot de données en quelque chose qui peut être itéré. Le passage d'un itérable à next() renverra l'élément suivant dans l'itérateur. Dans notre cas, next renverra un lot de 32 images et paires d'étiquettes.

In [ ]:
# Visualize training images from the training data batch
train_images, train_labels = next(train_data.as_numpy_iterator())
show_25_images(train_images, train_labels)

In [ ]:
# Visualize validation images from the validation data batch
val_images, val_labels = next(val_data.as_numpy_iterator())
show_25_images(val_images, val_labels)

# Création et formation d'un modèle

Maintenant que nos données sont prêtes, préparons leur modélisation. Nous allons utiliser un modèle existant de TensorFlow Hub.

TensorFlow Hub est une ressource où vous pouvez trouver des modèles d'apprentissage automatique pré-entraînés pour le problème sur lequel vous travaillez.
L'utilisation d'un modèle d'apprentissage automatique pré-entraîné est souvent appelée apprentissage par transfert.

Pourquoi utiliser un modèle pré-entraîné ?
Construire un modèle d'apprentissage automatique et l'entraîner sur des lots à partir de zéro peut être coûteux et prendre du temps. L'apprentissage par transfert permet d'éliminer certains de ces inconvénients en prenant ce qu'un autre modèle a appris et en utilisant ces informations pour votre propre problème.

# Comment choisir un modèle ?

Puisque nous savons que notre problème est la classification d'images (classification de différentes races de chiens), nous pouvons naviguer sur la page du Hub TensorFlow en fonction de notre domaine de problème (image). Nous commençons par choisir le domaine problématique de l'image, puis nous pouvons le filtrer par sous-domaines, dans notre cas, la classification d'images.

Ce faisant, nous obtenons une liste de différents modèles pré-entraînés que nous pouvons appliquer à notre tâche. En cliquant sur l'un d'eux, nous obtenons des informations sur le modèle ainsi que des instructions pour l'utiliser.

Par exemple, en cliquant sur le modèle mobilenet_v2_130_224, nous apprenons que ce modèle prend en entrée des images de la forme 224, 224. Il est également indiqué que le modèle a été formé dans le domaine de la classification d'images.

# Construction d'un modèle

Avant de construire un modèle, nous devons définir quelques éléments :

La forme d'entrée (images, sous la forme de tenseurs) de notre modèle.
La forme de sortie (étiquettes d'images, sous forme de tenseurs) de notre modèle.
L'URL du modèle que nous voulons utiliser.
Ces éléments constituent une pratique standard, quel que soit le modèle d'apprentissage automatique que vous utilisez. Et comme nous utilisons TensorFlow, tout sera sous la forme de tenseurs.

Remarque : pour utiliser les URL de TensorFlow Hub sur Kaggle, vous devez activer "internet" dans les paramètres du noyau.

In [ ]:
# Setup input shape to the model
INPUT_SHAPE = [None, IMG_SIZE, IMG_SIZE, 3] # batch, height, width, colour channels

# Setup output shape of the model
OUTPUT_SHAPE = len(unique_breed) # number of unique labels

# Setup model URL from TensorFlow Hub
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4"

# Data augmentation

https://www.tensorflow.org/tutorials/images/data_augmentation  - 
https://towardsdatascience.com/exploring-image-data-augmentation-with-keras-and-tensorflow-a8162d89b844

In [ ]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])

#  Construction du  modèle (suite)

Maintenant, nous avons les entrées, les sorties et le modèle que nous utilisons prêts à être utilisés. Nous pouvons commencer à les assembler

Il existe de nombreuses façons de construire un modèle dans TensorFlow, mais l'une des meilleures façons de commencer est d'utiliser l'API Keras.

Définir un modèle d'apprentissage profond dans Keras peut être aussi simple que de dire "voici les couches du modèle, la forme d'entrée et la forme de sortie, allons-y !".

Sachant cela, créons une fonction qui :

Prend la forme d'entrée, la forme de sortie et l'URL du modèle que nous avons choisi comme paramètres.
Définit les couches d'un modèle Keras de manière séquentielle (faire ceci d'abord, puis ceci, puis cela).
Compile le modèle (indique comment il doit être évalué et amélioré).
Construit le modèle (lui indique le type de forme d'entrée qu'il recevra).
Renvoie le modèle.


In [ ]:
# Create a function which builds a Keras model
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
  print("Building model with:", MODEL_URL)

  # Setup the model layers
  model = tf.keras.Sequential([
    resize_and_rescale,
    data_augmentation,
    hub.KerasLayer(MODEL_URL), # Layer 1 (input layer), TensorFlow Hub layer, requires Kaggle internet setting turned on
    tf.keras.layers.Dense(units=OUTPUT_SHAPE, 
                          activation="softmax") # Layer 2 (output layer)
  ])

  # Compile the model
  model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(), # Our model wants to reduce this (how wrong its guesses are)
      optimizer=tf.keras.optimizers.Adam(), # A friend telling our model how to improve its guesses
      metrics=["accuracy"] # We'd like this to go up
  )

  # Build the model
  model.build(INPUT_SHAPE) # Let the model know what kind of inputs it'll be getting
  
  return model

# Configuration des couches du modèle

Il y a deux façons de faire cela dans Keras, l'API fonctionnelle et séquentielle. Nous avons utilisé l'API séquentielle.

Laquelle devriez-vous utiliser ?

La documentation de Keras indique que l'API fonctionnelle est la meilleure solution pour définir des modèles complexes, mais l'API séquentielle (une pile linéaire de couches) est parfaitement adaptée pour commencer, ce que nous faisons.

La première couche que nous utilisons est le modèle de TensorFlow Hub (hub.KerasLayer(MODEL_URL). Notre première couche est donc en fait un modèle entier (plusieurs autres couches). Cette couche d'entrée prend nos images et y trouve des modèles basés sur les modèles que mobilenet_v2_130_224 a trouvés.

La couche suivante (tf.keras.layers.Dense()) est la couche de sortie de notre modèle. Elle rassemble toutes les informations découvertes dans la couche d'entrée et les restitue sous la forme que nous recherchons, 120 (le nombre d'étiquettes uniques que nous avons).

Le paramètre activation="softmax" indique à la couche de sortie que nous aimerions attribuer une valeur de probabilité à chacune des 120 étiquettes, quelque part entre 0 et 1. Plus la valeur est élevée, plus le modèle pense que l'image d'entrée devrait avoir cette étiquette. Si nous travaillions sur un problème de classification binaire, nous utiliserions la fonction d'activation "sigmoïde".

# Compilation du modèle

La meilleure façon de l'expliquer est de raconter une histoire.

Disons que vous participez aux championnats internationaux de descente de collines. Vous commencez en haut d'une colline et votre objectif est d'arriver en bas de la colline. Le problème est que vous avez les yeux bandés.

Heureusement, votre ami Adam est en bas de la colline et vous donne des instructions pour descendre.

En bas de la colline, il y a un juge qui évalue vos progrès. Il sait où tu dois arriver, alors il compare ta performance à celle que tu es censé avoir. Leur comparaison est la façon dont vous êtes noté.

Transférer ceci à la terminologie de **model.compile()** :

**loss** - La hauteur de la colline est la fonction de perte, l'objectif du modèle est de la minimiser, arriver à 0 (le bas de la colline) signifie que le modèle apprend parfaitement.

**Optimiseur** - Votre ami Adam est l'optimiseur, c'est lui qui vous dit comment franchir la colline (réduire la fonction de perte) en fonction de ce que vous avez fait jusqu'à présent. Il s'appelle Adam car l'optimiseur Adam est un grand général qui donne de bons résultats sur la plupart des modèles. D'autres optimiseurs incluent RMSprop et Stochastic Gradient Descent.

**métriques** - C'est le spectateur en bas de la colline qui évalue vos performances. Dans notre cas, il s'agit de la précision avec laquelle notre modèle prédit l'étiquette correcte de l'image.
Construction du modèle

Nous utilisons **model.build()** chaque fois que nous utilisons une couche de TensorFlow Hub pour indiquer à notre modèle la forme d'entrée à laquelle il peut s'attendre.

Dans ce cas, la forme d'entrée est [None, IMG_SIZE, IMG_SIZE, 3] ou [None, 224, 224, 3] ou [batch_size, img_height, img_width, color_channels].

La taille du lot est laissée à None car elle est déduite des données que nous transmettons au modèle. Dans notre cas, elle sera de 32 puisque c'est ce que nous avons configuré pour nos lots de données.

Maintenant que nous avons parcouru chaque section de la fonction, utilisons-la pour créer un modèle.

Nous pouvons appeler **summary()** sur notre modèle pour avoir une idée de ce à quoi il ressemble.

In [ ]:
# Create a model and check its details
model = create_model()
model.summary()

Les paramètres **non entraînables sont les modèles appris par mobilenet_v2_130_224** et les **paramètres entraînables sont ceux de la couche dense** que nous avons ajoutée.

Cela signifie que la majeure partie de l'information dans notre modèle a déjà été apprise et nous allons la prendre et l'adapter à notre propre problème.

# Création de callbacks (choses pour aider notre modèle)
Nous avons un modèle prêt à l'emploi, mais avant de l'entraîner, nous devons créer quelques fonctions de rappel.

Les callbacks sont des fonctions d'aide qu'un modèle peut utiliser pendant la formation pour effectuer des opérations telles que la sauvegarde de la progression d'un modèle, la vérification de la progression d'un modèle ou l'arrêt anticipé de la formation si le modèle ne s'améliore plus.

Les deux callbacks que nous allons ajouter sont un callback TensorBoard et un callback d'arrêt précoce.

*Rappel de TensorBoard*
TensorBoard permet de fournir un moyen visuel de **surveiller la progression de votre modèle pendant et après la formation.**

Il peut être utilisé directement dans un carnet de notes pour suivre les mesures de performance d'un modèle telles que la perte et la précision.

Pour configurer un callback TensorBoard et afficher TensorBoard dans un notebook, nous devons faire trois choses :

* Charger l'extension de notebook TensorBoard.
* Créer un callback TensorBoard capable d'enregistrer les logs dans un répertoire et de les transmettre à la fonction fit() de notre modèle.
* Visualiser les logs d'entraînement de notre modèle en utilisant la fonction magique %tensorboard (nous le ferons plus tard).

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import datetime

# Create a function to build a TensorBoard callback
def create_tensorboard_callback():
  # Create a log directory for storing TensorBoard logs
  logdir = os.path.join("drive/My Drive/Data/logs",
                        # Make it so the logs get tracked whenever we run an experiment
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tf.keras.callbacks.TensorBoard(logdir)

# Rappel d'arrêt anticipé

L'arrêt précoce permet d'**éviter le surajustement en arrêtant un modèle lorsqu'une certaine métrique d'évaluation cesse de s'améliorer.** Si un modèle s'entraîne trop longtemps, il peut réussir à trouver des modèles dans un certain ensemble de données et ne pas être capable d'utiliser ces modèles dans un autre ensemble de données qu'il n'a pas vu auparavant (il ne peut pas généraliser).

Cela revient à dire à notre modèle : "continuez à trouver des modèles jusqu'à ce que la qualité de ces modèles commence à diminuer".

In [ ]:
# Create early stopping (once our model stops improving, stop training)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=3) # stops after 3 rounds of no improvements

# Entraînement d'un modèle (sur un sous-ensemble de données)

Notre premier modèle ne sera entraîné que sur 1000 images. Ou formé sur 800 images, puis validé sur 200 images, soit 1000 images au total ou environ 10% des données totales.

Nous faisons cela pour nous assurer que tout fonctionne. Et si c'est le cas, nous pouvons passer à la vitesse supérieure plus tard et nous entraîner sur l'ensemble des données d'entraînement.

Le dernier paramètre que nous définirons avant l'entraînement est NUM_EPOCHS (également connu sous le nom de nombre d'époques).

**NUM_EPOCHS** définit le nombre de passages des données que nous souhaitons que notre modèle effectue. Un passage équivaut à ce que notre modèle essaie de trouver des motifs dans chaque image de chien et de voir quels motifs sont liés à chaque étiquette.

Si NUM_EPOCHS=1, le modèle ne regardera les données qu'une seule fois et obtiendra probablement de mauvais résultats car il n'aura pas la possibilité de se corriger. C'est comme si vous participiez à un championnat international de descente de colline et que votre ami Adam ne pouvait vous donner qu'une seule instruction pour descendre la colline.

Quelle est la valeur de NUM_EPOCHS ?

C'est difficile à dire. 10 pourrait être un bon début, mais 100 aussi. C'est l'une des raisons pour lesquelles nous avons créé un rappel d'arrêt anticipé. La configuration de l'arrêt anticipé signifie que si nous définissons NUM_EPOCHS à 100 mais que notre modèle cesse de s'améliorer après 22 époques, il arrêtera la formation.

En plus de cela, vérifions rapidement si nous utilisons toujours un GPU.

In [ ]:
# Check again if GPU is available (otherwise computing will take a looooonnnnggggg time)
print("GPU", "available (OUUUUIIII!!!!)" if tf.config.list_physical_devices("GPU") else "NONNNNNN!!!!!")

In [ ]:
# How many rounds should we get the model to look through the data?
NUM_EPOCHS = 100

Créons une fonction simple qui entraîne un modèle. La fonction va :

* Créer un modèle en utilisant create_model().
* Configurer un callback TensorBoard en utilisant create_tensorboard_callback() (nous le faisons ici pour qu'il crée un répertoire de log avec la date et l'heure actuelles).
* Appelez la fonction fit() sur notre modèle en lui passant les données d'entraînement, les données de validation, le nombre d'époques d'entraînement et les callbacks que nous souhaitons utiliser.

In [ ]:
# Build a function to train and return a trained model
def train_model():
  """
  Trains a given model and returns the trained version.
  """
  # Create a model
  model = create_model()

  # Create new TensorBoard session everytime we train a model
  tensorboard = create_tensorboard_callback()

  # Fit the model to the data passing it the callbacks we created
  model.fit(x=train_data,
            epochs=NUM_EPOCHS,
            validation_data=val_data,
            validation_freq=1, # check validation metrics every epoch
            callbacks=[tensorboard, early_stopping])
  
  return model

In [ ]:
# Fit the model to the data
model = train_model()

 Il semble que notre modèle puisse être surajusté (il obtient de bien meilleurs résultats sur l'ensemble d'apprentissage que sur l'ensemble de validation), quels sont les moyens d'empêcher le surajustement du modèle ? Indice : cela peut impliquer de chercher quelque chose comme "comment éviter l'overfitting dans un modèle d'apprentissage profond".

Remarque : au départ, l'overfitting est une bonne chose. Cela signifie que notre modèle est en train d'apprendre quelque chose.

Vérification des journaux de TensorBoard
Maintenant que notre modèle a été formé, nous pouvons visualiser ses performances en vérifiant les journaux de TensorBoard.

La fonction magique de TensorBoard (%tensorboard) accède au répertoire de logs que nous avons créé plus tôt et visualise son contenu.

In [ ]:
%tensorboard --logdir logs

Grâce à notre callback early_stopping, le modèle a arrêté l'apprentissage après environ 26 époques (dans mon cas, le vôtre pourrait être légèrement différent). Cela est dû au fait que la précision de la validation n'a pas progressé pendant 3 époques.

Mais la bonne nouvelle est que nous pouvons clairement voir que notre modèle apprend quelque chose. La précision de validation est passée à 65% en seulement quelques minutes.

Cela signifie que si nous augmentons le nombre d'images, nous pouvons espérer voir la précision augmenter.

**Avant de passer à l'échelle supérieure et de nous entraîner sur davantage de données, voyons d'autres façons d'évaluer notre modèle. En effet, bien que la précision soit un bon indicateur de l'efficacité de notre modèle**, il serait encore mieux de pouvoir le voir en action.

Pour faire des prédictions avec un modèle entraîné, il suffit d'appeler predict() sur celui-ci et de lui transmettre des données dans le même format que celui sur lequel le modèle a été entraîné.

In [ ]:
# Make predictions on the validation data (not used to train on)
predictions = model.predict(val_data, verbose=1) # verbose shows us how long there is to go
predictions

In [ ]:
# Check the shape of predictions
predictions.shape

La réalisation de prédictions avec notre modèle renvoie un tableau avec une valeur différente pour chaque étiquette.

Dans ce cas, faire des prédictions sur les données de validation (200 images) renvoie un tableau (prédictions) de tableaux, chacun contenant 120 valeurs différentes (une pour chaque race de chien unique).

Ces différentes valeurs sont les probabilités ou la probabilité que le modèle ait prédit qu'une certaine image était une certaine race de chien. Plus la valeur est élevée, plus le modèle pense qu'une image donnée est une race de chien spécifique.

Voyons comment convertir un tableau de probabilités en une étiquette réelle.

In [ ]:
# First prediction
print(predictions[0])
print(f"Max value (probability of prediction): {np.max(predictions[0])}") # the max probability value predicted by the model
print(f"Sum: {np.sum(predictions[0])}") # because we used softmax activation in our model, this will be close to 1
print(f"Max index: {np.argmax(predictions[0])}") # the index of where the max value in predictions[0] occurs
print(f"Predicted label: {unique_breed[np.argmax(predictions[0])]}") # the predicted label

C'est bien d'avoir cette information, mais ce serait encore mieux si nous pouvions comparer une prédiction à son véritable label et à l'image originale.

Pour nous aider, nous allons d'abord construire une petite fonction pour convertir les probabilités de prédiction en étiquettes prédites.

Remarque : Les probabilités de prédiction sont également connues sous le nom de niveaux de confiance.

In [ ]:
# Turn prediction probabilities into their respective label (easier to understand)
def get_pred_label(prediction_probabilities):
  """
  Turns an array of prediction probabilities into a label.
  """
  return unique_breed[np.argmax(prediction_probabilities)]

# Get a predicted label based on an array of prediction probabilities
pred_label = get_pred_label(predictions[0])
pred_label

Rappelez-vous, le modèle n'a pas été entraîné sur les données de validation, pendant la fonction fit(), il a seulement utilisé les données de validation pour s'évaluer. Nous pouvons donc utiliser les images de validation pour comparer visuellement les prédictions de notre modèle avec les étiquettes de validation.

Puisque nos données de validation (val_data) sont sous forme de lot, pour obtenir une liste d'images et d'étiquettes de validation, nous devrons les débloquer (en utilisant unbatch()) et ensuite les transformer en itérateur en utilisant as_numpy_iterator().


In [ ]:
# Create a function to unbatch a batched dataset
def unbatchify(data):
  """
  Takes a batched dataset of (image, label) Tensors and returns separate arrays
  of images and labels.
  """
  images = []
  labels = []
  # Loop through unbatched data
  for image, label in data.unbatch().as_numpy_iterator():
    images.append(image)
    labels.append(unique_breed[np.argmax(label)])
  return images, labels

# Unbatchify the validation data
val_images, val_labels = unbatchify(val_data)
val_images[0], val_labels[0]

Maintenant, nous avons des moyens d'obtenir :

* Des étiquettes de prédiction
* Étiquettes de validation (étiquettes de vérité)
* Images de validation
* Créons quelques fonctions pour rendre tout cela un peu plus visuel.

Plus précisément, nous voulons être en mesure de visualiser une image, son étiquette prédite et son étiquette réelle (true label).

La première fonction que nous allons créer va :

* Prendre un tableau de probabilités de prédiction, un tableau d'étiquettes réelles, un tableau d'images et un nombre entier.
* Convertir les probabilités de prédiction en une étiquette prédite.
* Tracer l'étiquette prédite, sa probabilité prédite, l'étiquette de vérité et l'image cible sur un seul graphique.

In [ ]:
def plot_pred(prediction_probabilities, labels, images, n=1):
  """
  View the prediction, ground truth label and image for sample n.
  """
  pred_prob, true_label, image = prediction_probabilities[n], labels[n], images[n]
  
  # Get the pred label
  pred_label = get_pred_label(pred_prob)
  
  # Plot image & remove ticks
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])

  # Change the color of the title depending on if the prediction is right or wrong
  if pred_label == true_label:
    color = "green"
  else:
    color = "red"

  plt.title("{} {:2.0f}% ({})".format(pred_label,
                                      np.max(pred_prob)*100,
                                      true_label),
                                      color=color)

In [ ]:
# View an example prediction, original image and truth label
plot_pred(prediction_probabilities=predictions,
          labels=val_labels,
          images=val_images)

Comme nous travaillons sur un problème multi-classes (120 races de chiens différentes), il serait également intéressant de voir quelles autres prédictions notre modèle fait. Plus précisément, si notre modèle prédit une certaine étiquette avec une probabilité de 24 %, que prédit-il d'autre ?

Construisons une fonction pour le démontrer. Cette fonction va :
**Prendre en entrée un tableau de probabilités de prédiction, un tableau d'étiquettes de vérité terrain et un nombre entier./
Trouver l'étiquette prédite en utilisant get_pred_label()./
Trouver les 10 premiers :
Indices de probabilités de prédiction - 
Valeurs des probabilités de prédiction - 
Étiquettes de prédiction /. 
Tracez les 10 premières valeurs de probabilité de prédiction et les étiquettes, en colorant la véritable étiquette en vert.**

In [ ]:
def plot_pred_conf(prediction_probabilities, labels, n=1):
  """
  Plots the top 10 highest prediction confidences along with
  the truth label for sample n.
  """
  pred_prob, true_label = prediction_probabilities[n], labels[n]

  # Get the predicted label
  pred_label = get_pred_label(pred_prob)

  # Find the top 10 prediction confidence indexes
  top_10_pred_indexes = pred_prob.argsort()[-10:][::-1]
  # Find the top 10 prediction confidence values
  top_10_pred_values = pred_prob[top_10_pred_indexes]
  # Find the top 10 prediction labels
  top_10_pred_labels = unique_breed[top_10_pred_indexes]

  # Setup plot
  top_plot = plt.bar(np.arange(len(top_10_pred_labels)), 
                     top_10_pred_values, 
                     color="grey")
  plt.xticks(np.arange(len(top_10_pred_labels)),
             labels=top_10_pred_labels,
             rotation="vertical")

  # Change color of true label
  if np.isin(true_label, top_10_pred_labels):
    top_plot[np.argmax(top_10_pred_labels == true_label)].set_color("green")
  else:
    pass

In [ ]:
plot_pred_conf(prediction_probabilities=predictions,
               labels=val_labels,
               n=9)

In [ ]:
# Let's check a few predictions and their different values
i_multiplier = 0
num_rows = 3
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(5*2*num_cols, 5*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_pred(prediction_probabilities=predictions,
            labels=val_labels,
            images=val_images,
            n=i+i_multiplier)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_pred_conf(prediction_probabilities=predictions,
                labels=val_labels,
                n=i+i_multiplier)
plt.tight_layout(h_pad=1.0)
plt.show()

https://www.kaggle.com/mrdbourke/tensorflow-2-x-tensorflow-hub-end-to-end-example